In [ ]:

## Running in Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Unfettered-b/Asgard_Protein_annotation/blob/main/search_cds.ipynb)



!apt-get -qq install git-lfs
!git clone https://github.com/Unfettered-b/Asgard_Protein_annotation.git
%cd Asgard_Protein_annotation
!git lfs install
!git lfs pull
!ls -lh data/

In [ ]:
import pandas as pd


completeness = 50
# cds_file = f"/home/anirudh/genomes/complete{completeness}/Proteins_genomes_cp{completeness}.csv"
cds_file = f"data/Proteins_genomes_cp{completeness}.csv"


cds_db = pd.read_csv(cds_file)
# cds_db['header'] = cds_db['header'].str.lower()
print(f"Number of entries in db: {len(cds_db)}")
print(f"Number of unique species in db: {cds_db['organism_name'].nunique()}")

search_term = input("What protein are you looking for?: ").strip()
selected_db = cds_db[cds_db['header'].str.contains(search_term, case=False, na=False)]
print(f"{len(selected_db)} results obtained for the search {search_term}")

print(selected_db.head(10))